In [12]:
#to form Stack Class
class StackClass:

    def __init__(self, itemlist=[]):
        self.items = itemlist

    def isEmpty(self):
        if self.items == []:
            return True
        else:
            return False

    def peek(self):
        return self.items[-1:][0]

    def pop(self):
        return self.items.pop()

    def push(self, item):
        self.items.append(item)
        return 0


In [13]:
# Infix to postfix Conversion of the expression given
def infixtopostfix(infixexpr):

    s=StackClass()
    outlst=[]
    prec={}
    # this is the precedence order
    prec['*']=3
    prec['|']=2
    prec['.']=1
    prec['(']=1
    oplst=['*','|','.']

    tokenlst=infixexpr.split()
    for token in tokenlst:
        if token in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' or token in '0123456789':
            outlst.append(token)

        elif token == '(':
            s.push(token)

        elif token == ')':
            topToken=s.pop()
            while topToken != '(':
                outlst.append(topToken)
                topToken=s.pop()
        else:
            while (not s.isEmpty()) and (prec[s.peek()] >= prec[token]):
                #print token
                outlst.append(s.pop())
                #print outlst

            s.push(token)
            print (s.peek())

    while not s.isEmpty():
        opToken=s.pop()
        outlst.append(opToken)
        #print outlst
    return outlst
    #return " ".join(outlst)

In [14]:
# extract the keys from the expression as well as the operators
import re
postfix = infixtopostfix("( A | B )")
regex=''.join(postfix)
# | is for OR, * for Kleene Closure, . is for concatenation
keys=list(set(re.sub('[^A-Za-z0-9]+', '', regex)+'e'))
# e stands for epsilon

|


In [15]:
print(keys)
print(len(keys))
print(postfix)

['e', 'A', 'B']
3
['A', 'B', '|']


In [16]:
# Regex to NFA
s=[];stack=[];start=0;end=1

counter=-1;c1=0;c2=0

for i in regex:
    if i in keys:
        counter=counter+1;
        c1=counter;
        counter=counter+1;
        c2=counter;
        s.append({});
        s.append({})
        stack.append([c1,c2])
        
        s[c1][i]=c2
    elif i=='*':
        r1,r2=stack.pop()
        counter=counter+1;
        c1=counter;
        counter=counter+1;
        c2=counter;
        s.append({});
        s.append({})
        stack.append([c1,c2])
        s[r2]['e']=[r1,c2];s[c1]['e']=[r1,c2]
        if start==r1:
            start=c1 
        if end==r2:
            end=c2 
    elif i=='.':
        r11,r12=stack.pop()
        r21,r22=stack.pop()
        r12 = r11
        r11 = r22
        stack.append([r21,r12])
        elem = s[r12]
        del s[r12]
        for key in elem.keys():
            s[r11][key] = elem.get(key)-1
        counter = counter - 1
        if start==r11:
            start=r21 
        if end==r22:
            end=r12 
    else:
        counter=counter+1;
        c1=counter;
        counter=counter+1;
        c2=counter;
        s.append({});
        s.append({})
        r11,r12=stack.pop()
        r21,r22=stack.pop()
        stack.append([c1,c2])
        s[c1]['e']=[r21,r11]; s[r12]['e']=c2; s[r22]['e']=c2
        if start==r11 or start==r21:
            start=c1 
        if end==r22 or end==r12:
            end=c2

print(keys)
print(start)
print(end)
print(s)

['e', 'A', 'B']
4
5
[{'A': 1}, {'e': 5}, {'B': 3}, {'e': 5}, {'e': [0, 2]}, {}]


In [17]:
# Convert the above s list into a matrix
import pandas as pd
import numpy as np

arr_mat = np.full((end + 1, len(keys)), [-1])
df_trans = pd.DataFrame(arr_mat, columns = keys, index = range(0, end+1), dtype = 'object')
#print(df_trans)
i = 0
for elem in s:
    for key, value in elem.items():
        #print(key, value)
        df_trans.set_value(i, key, value)
    i = i+1
print(df_trans)

        e   A   B
0      -1   1  -1
1       5  -1  -1
2      -1  -1   3
3       5  -1  -1
4  [0, 2]  -1  -1
5      -1  -1  -1


In [18]:
#Convert NFA to DFA, get all the e-closures
def get_e_closure(state):
    stack_states = StackClass(list(state))
    e_closure = state
    #print(e_closure)
    while not(stack_states.isEmpty()):
        top_ele = stack_states.pop()
        e_states = df_trans.get_value(top_ele, 'e')
        #print(isinstance(e_states, list))
        if e_states != -1 and isinstance(e_states, list):
            for e_state in e_states:
                #print(e_state)
                if e_state not in e_closure:
                    e_closure.append(e_state)
                    stack_states.push(e_state)
        elif e_states!=-1:
            if e_states not in e_closure:
                e_closure.append(e_states)
                stack_states.push(e_states)
    print(e_closure)
    return e_closure
get_e_closure([4, 5])

[4, 5, 0, 2]


[4, 5, 0, 2]

In [19]:
#Construct DFA from NFA, Procedure
state0 = get_e_closure(list(map(int,str(start))))
unmarked_states = []
all_dfa_states = []
all_dfa_states.append(state0)
unmarked_states.append(state0)
print(unmarked_states)
keys_dfa=list(set(re.sub('[^A-Za-z0-9]+', '', regex)))
print(keys_dfa)
arr_mat = np.full((1, len(keys_dfa)), [-1])
DFA_trans = pd.DataFrame(arr_mat, columns = keys_dfa) 
print(DFA_trans)

[4, 0, 2]
[[4, 0, 2]]
['A', 'B']
   A  B
0 -1 -1


In [20]:
index = -1
while unmarked_states:
    index = index + 1
    s1 = unmarked_states.pop(0)
    for key in keys_dfa:
        inp_for_closure = []
        for state in s1:
            val_df = df_trans.get_value(state, key)
            if val_df!=-1:
                inp_for_closure.append(val_df)
        #print(inp_for_closure)
        s2 = get_e_closure(inp_for_closure)
        if s2 not in all_dfa_states:
            all_dfa_states.append(s2)
            unmarked_states.append(s2)
            DFA_trans = DFA_trans.append(pd.DataFrame(arr_mat, columns = keys_dfa), ignore_index = True)
        DFA_trans.set_value(index, key, all_dfa_states.index(s2))
print(DFA_trans)

[1, 5]
[3, 5]
[]
[]
[]
[]
[]
[]
   A  B
0  1  2
1  3  3
2  3  3
3  3  3


In [ ]:
-----------------END-------------------